<a href="https://colab.research.google.com/github/kohnatsuhara/youtube_scraping/blob/googlecolab/auto_youtube_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 広告情報取得

In [ ]:
# 必要なパッケージ
!pip3 install openpyxl xlwt
!pip3 install selenium
!pip install pyvirtualdisplay
!apt-get update
!apt-get install -y xvfb

In [6]:
import pyvirtualdisplay

# 仮想ディスプレイ環境を構築
display = pyvirtualdisplay.Display(visible=0, size=(1024, 768))
display.start()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu 

In [15]:
# googledriveをマウント
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [23]:
# マウントしたgoogledrive内のconfigファイルを読み込む
# config.py内のYOUTUBE_API_KEYを追記しておく
import sys
sys.path.append("/content/drive/MyDrive/youtube_scraping-main/")
from config import *

In [8]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import re
import time
import pandas as pd

In [9]:
def click_start(driver):
    """
    広告を再生するために動画の再生ボタンを押す
    """
    xpath = '//*[@id="movie_player"]/div[8]/button'
    element = driver.find_elements(by=By.XPATH, value=xpath)
    try:
        element[0].click()
        print("click")
    except Exception as e:
        print(e)

def get_video_info(videoID):
    """
    指定したvideoIDの詳細情報を取得
    """

    params = {
        "id": videoID,
        "key": YOUTUBE_API_KEY,
        "part": 'snippet,statistics,status,contentDetails,topicDetails',
        # "fields": "id,snippet,statistics,status",
    }

    r = requests.get(URL_VIDEOS, params=params)
    result_dic = {}
    result_dic["viewCount"] = r.json()["items"][0]["statistics"].get('viewCount',"不明")# 再生回数
    result_dic["commentCount"] = r.json()["items"][0]["statistics"].get('commentCount',"不明")# コメント数
    result_dic["likeCount"] = r.json()["items"][0]["statistics"].get('likeCount',"不明")# いいねの数
    result_dic["tags"] = ",".join(r.json()["items"][0]["snippet"].get("tags","不明"))# タグ
    result_dic["description"] = r.json()["items"][0]["snippet"].get("description","不明")# 動画詳細
    result_dic["title"] = r.json()["items"][0]["snippet"].get("title","不明")#動画タイトル
    result_dic["channelTitle"] = r.json()["items"][0]["snippet"].get("channelTitle","不明")# チャンネル名
    return result_dic


In [11]:
# 検索ワード
video_search_words = "ポケモン"

# 1動画ごとに取得するCM数（動画再生を繰り返して取得するCM数）
# 繰り返し再生しても大抵一種類のCMしか再生されない
cm_num = 5
# 情報取得する動画数
video_num = 10

params = {
    "key": YOUTUBE_API_KEY,
    "part": 'snippet',
    "q":video_search_words,
    "order":'viewCount',
    "type":'video',
    "maxResults": video_num
    # "fields": "id,snippet,statistics,status",
}
r = requests.get(URL_SEARCH, params=params)

# top_bideoID = r.json()["items"][0]["id"]["videoId"]

result_df = pd.DataFrame()
for n,video in enumerate(r.json()["items"][:video_num]):

    options = Options()
    # linux上で動かすときは次の2つも実行してください。
    options.add_argument('--headless')
    options.add_argument('--incognito') #シークレットモード
    options.add_argument("--no-sandbox") # サンドボックスモードを無効にするオプション
    options.add_argument("--disable-dev-shm-usage") # /dev/shmへのアクセスを禁止

    driver = webdriver.Chrome(options=options)
    videoID = video['id']["videoId"]
    print(n,videoID)

    url = f"https://www.youtube.com/watch?v={videoID}"
    driver.get(url)
    url_list = []
    count_num = 0
    print("開始")
    while True:
        driver.refresh() # ページの更新
        time.sleep(5)
        click_start(driver) #動画再生（広告の割り込み）
        time.sleep(3)

        # 広告URLが格納されているxpath
        xpath = '//*[@id="companion"]/top-banner-image-text-icon-buttoned-layout-view-model/div[2]/ad-button-view-model/a'
        result = driver.find_elements(by=By.XPATH, value=xpath)

        # 取得した要素のHTMLを表示
        for element in result:
            # url_search_list = re.findall('adurl=(.*)', element.get_attribute("href"))
            url_search_list = element.get_attribute("href")
            if len(url_search_list) > 0:
                url_list.append(url_search_list)
                url_num = len(url_list)
                count_num = 0

        if (len(url_list)>cm_num) or (count_num==9):
            break
        count_num += 1

    url_list = list(set(url_list))
    video_info_dic = get_video_info(videoID)
    video_info_dic["koukoku"]= ",".join(url_list)
    a = pd.Series(video_info_dic, name=f"rank{n}")

    result_df = pd.concat([result_df, a.to_frame().T])
    # driverを終了する
    driver.quit()

0 oP-CWIgeY9E
開始
click
click
click
click
click
click
click
click
click
click
1 zlRDOj-3EpA
開始
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
2 BoZ0Zwab6Oc
開始
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
3 qYllGk_5gl0
開始
click
click
click
click
click
click
click
click
click
click
4 23g5HBOg3Ic
開始
click
click
click
click
click
click
click
click
click
click
5 diIQBFaBNHU
開始
click
click
click
click
click
click
click
click
click
click
6 L6GiPS_KZos
開始
click
click
click
click
click
click
click
click
click
click
click
click
7 Cn34P8-tssE
開始
list index out of range
list index out of range
list index out of range


In [16]:
# マウントしたgoogledriveに保存する
result_df.to_excel("./drive/MyDrive/result.xlsx")

In [25]:
result_df.shape

(10, 8)